In [ ]:
import time
import sys

sys.path.append('/home/cjw/Code/Unet')

from skimage.io import imread
from matplotlib import pyplot as plt
import numpy as np

import unet
%matplotlib inline

In [ ]:
import tensorflow as tf

In [ ]:
params = dict()

params['width'] = 256
params['height'] = 256
params['depth'] = 96
params['nchannels'] = 1
params['channels'] = [1]
params['nepochs'] = [1]
params['batchsize'] = [2]
params['learning_rate'] = 0.001
params['restore'] = False
params['latent_size'] = 1
params['enc_sizes'] = [[16,3], [32, 3], [64, 3], [128, 3]]
params['dec_sizes'] = [[64,3], [32, 3], [16, 3], [params['nchannels'], 3]]
params['droprate'] = 0
params['stdev'] = 0.08

In [ ]:

tf.reset_default_graph()
u = unet.unet3d(params)


In [ ]:
u.read_images('/media/cjw/Data/Nuclei.tif')
u.x = u.x[:,:,:,:,:]
u.x.shape

In [ ]:

images = tf.placeholder(tf.float32, (None, 256, 256, 96, params['nchannels']))
masks = tf.placeholder(tf.float32, (None, 256, 256, 96, params['nchannels']))

In [ ]:
enc = u.create_encoder(images, True)

In [ ]:
d = u.create_decoder()
u.create_loss(masks)
u.create_opt()

In [ ]:
sess = tf.Session()

In [ ]:
sess.run(tf.global_variables_initializer())

In [ ]:
for i in range(1000):
    nr = np.random.randint(0,36, 8)
    bx = np.expand_dims(u.x[nr,:,:,:,0], -1)
    bm = np.expand_dims(u.x[nr,:,:,:,1], -1)
    _, res, err = sess.run([u.opt, u.decoder_sigmoid, u.loss], feed_dict={images:bx, masks:bm})
    if i % 20 == 0:
        print(err)
        plt.figure(figsize=(8,2))
        plt.subplot(1,3,1)
        plt.imshow(res[0,:,:,32,0])
        plt.subplot(1,3,2)
        plt.imshow(u.x[nr[0],:,:,32,1])
        plt.subplot(1,3,3)
        plt.imshow(u.x[nr[0],:,:,32,0])
        plt.show()

In [ ]:
res.shape, u.x[nr,:,:,32,1].shape

In [ ]:
u.create_loss(masks)
ss = sess.run(u.loss, feed_dict={images:yy, masks:bb})

In [ ]:
ss.shape
plt.imshow(ss[0,:,:,44,0])

In [ ]:
rt = y[0,:,:,:,:]
from scipy.ndimage import rotate

res = rotate(rt, 45, (0, 1), order=2)

In [ ]:
plt.imshow(res[:,:,44,0])